# GPU Computing for Data Scientists
#### Using CUDA, Jupyter, PyCUDA, ArrayFire and Thrust


https://github.com/QuantScientist/Data-Science-ArrayFire-GPU

In [37]:
%reset -f
import pycuda
from pycuda import compiler
import pycuda.driver as drv
import pycuda.driver as cuda

# Make sure we have CUDA

In [38]:
drv.init()
print("%d device(s) found." % drv.Device.count())           
for ordinal in range(drv.Device.count()):
    dev = drv.Device(ordinal)
    print "Device #%d: %s" % (ordinal, dev.name())    

drv

1 device(s) found.
Device #0: GeForce GTX 1080


<module 'pycuda.driver' from '/usr/local/lib/python2.7/dist-packages/pycuda/driver.pyc'>

## Simple addition the GPU: compilation

In [39]:
import pycuda.autoinit
import numpy

from pycuda.compiler import SourceModule

ARR_SIZE=10

srcGPU = """
   __global__ void multGPU(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
"""

srcGPUModule = SourceModule(srcGPU)

print srcGPUModule

# Simple addition on the GPU: Host memory allocation

In [40]:
a = numpy.random.randn(ARR_SIZE).astype(numpy.float32)
b = numpy.random.randn(ARR_SIZE).astype(numpy.float32)

dest = numpy.zeros_like(a)
print dest

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


## Simple addition on the GPU: execution

In [41]:
multGPUFunc = srcGPUModule.get_function("multGPU")

print multGPUFunc

multGPUFunc(
        drv.Out(dest), drv.In(a), drv.In(b),
        block=(ARR_SIZE,1,1), grid=(1,1))

print dest

[ 1.55915952 -1.93428469  0.05227064 -0.81259573 -0.51810092  0.1830858
 -0.29488933  0.44019917  0.48972565  0.1234716 ]


In [42]:
a = numpy.random.randn(4,4)
a = a.astype(numpy.float32)

a_gpu = cuda.mem_alloc(a.nbytes)

cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)
  
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print a_doubled

[[-0.48068568 -0.55113399 -0.73733729  0.24453853]
 [-3.95931816  1.02043021 -1.63834941 -0.66162008]
 [-2.65286064 -3.01072693 -0.50203598  1.0968225 ]
 [-1.3004452   0.03259312  2.76198721 -2.37281108]]
